In [1]:
import os
import sys

import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from tfx.utils.dsl_utils import external_input
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen

In [2]:
# add project to the python path
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

os.environ["TFHUB_CACHE_DIR"] = '/Users/i854694/tmp/tfhub'

In [3]:
context = InteractiveContext(pipeline_root='../tfx/21Feb')

In [4]:
base_dir = os.getcwd()
data_dir = "../data/21Feb/"
examples = external_input(os.path.join(base_dir, data_dir))
example_gen = CsvExampleGen(input=examples)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 4
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ../tfx/21Feb/CsvExampleGen/examples/4, id: 4)]
        ))

In [5]:
for artifact in example_gen.outputs['examples'].get():
    print(artifact)

Artifact(type_name: Examples, uri: ../tfx/21Feb/CsvExampleGen/examples/4, id: 4)


In [6]:
from tfx.components.statistics_gen.component import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

context.show(statistics_gen.outputs['statistics'])

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-24T23_22_02.736385"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-24T23_22_02.736385"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-24T23_22_02.736385');
/Users/i854694/.virtualenvs/pipelines/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:236: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [7]:
from tfx.components.schema_gen.component import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

#context.show(schema_gen.outputs['schema'])

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-24T23_22_02.736385"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-24T23_22_02.736385"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-24T23_22_02.736385');
ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.schema_gen.component.SchemaGen


ExecutionResult(
    component_id: SchemaGen
    execution_id: 6
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: ../tfx/21Feb/SchemaGen/schema/6, id: 6)]
        ))

In [8]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',BYTES,required,,-
'company_response',STRING,required,,'company_response'
'date_received',STRING,required,,'date_received'
'date_sent_to_company',STRING,required,,'date_sent_to_company'
'issue',STRING,optional,single,'issue'
'product',STRING,required,,'product'
'state',STRING,optional,single,'state'
'sub_issue',STRING,optional,single,'sub_issue'
'sub_product',STRING,optional,single,'sub_product'


,Values
Domain,
'company_response',"'Closed', 'Closed with explanation', 'Closed with monetary relief', 'Closed with non-monetary relief', 'In progress', 'Untimely response'"
'date_received',"'2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-01-31', '2015-02-01', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-07', '2015-02-08', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-14', '2015-02-15', '2015-02-16', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-21', '2015-02-22', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-02-28', '2015-03-01', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-07', '2015-03-08', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-14', '2015-03-15', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19'"
'date_sent_to_company',"'2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-01-31', '2015-02-01', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-07', '2015-02-08', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-14', '2015-02-15', '2015-02-16', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-21', '2015-02-22', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-02-28', '2015-03-01', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-07', '2015-03-08', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-14', '2015-03-15', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19'"
'issue',"'APR or interest rate', 'Account opening, closing, or management', 'Account terms and changes', 'Adding money', 'Advertising and marketing', 'Advertising, marketing or disclosures', 'Application processing delay', 'Application, originator, mortgage broker', 'Applied for loan/did not receive money', 'Arbitration', 'Balance transfer', 'Balance transfer fee', 'Bankruptcy', 'Billing disputes', 'Billing statement', 'Can\'t contact lender', 'Can\'t repay my loan', 'Can\'t stop charges to bank account', 'Cash advance', 'Cash advance fee', 'Charged bank acct wrong day or amt', 'Charged fees or interest I didn\'t expect', 'Closing/Cancelling account', 'Communication tactics', 'Cont\'d attempts collect debt not owed', 'Convenience checks', 'Credit card protection / Debt protection', 'Credit decision / Underwriting', 'Credit determination', 'Credit line increase/decrease', 'Credit monitoring or identity protection', 'Credit reporting company\'s investigation', 'Customer service / Customer relations', 'Customer service/Customer relations', 'Dealing with my lender or servicer', 'Delinquent account', 'Deposits and withdrawals', 'Disclosure verification of debt', 'Disclosures', 'False statements or representation', 'Fees', 'Forbearance / Workout plans', 'Fraud or scam', 'Getting a loan', 'Identity theft / Fraud / Embezzlement', 'Improper contact or sharing of info', 'Improper use of my credit report', 'Incorrect exchange rate', 'Incorrect information on credit report', 'Incorrect/missing disclosures or info', 'Late fee', 'Lender repossessed or sold the vehicle', 'Loan modificatio

In [9]:
from tfx.components.example_validator.component import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-24T23_22_02.736385"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-24T23_22_02.736385"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-24T23_22_02.736385');
ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.example_validator.component.ExampleValidator


,Anomaly short description,Anomaly long description
Feature name,,
'state',Unexpected string values,Examples contain values missing from the schema: AS (<1%).
'issue',Unexpected string values,"Examples contain values missing from the schema: Excessive fees (<1%), Lender damaged or destroyed vehicle (<1%)."


In [39]:
transform_file = os.path.join(base_dir, '../components/transform.py')

In [40]:
from tfx.components.transform.component import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=transform_file)
context.run(transform)

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-24T23_22_02.736385"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-24T23_22_02.736385"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-24T23_22_02.736385');


INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: ../tfx/21Feb/Transform/transform_graph/26/.temp_path/tftransform_tmp/13954f4f047643dbae9f9a017c2da599/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: ../tfx/21Feb/Transform/transform_graph/26/.temp_path/tftransform_tmp/d05e51324ff541e2b0f53e99c6dcca11/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restor

ExecutionResult(
    component_id: Transform
    execution_id: 26
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: ../tfx/21Feb/Transform/transform_graph/26, id: 30)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ../tfx/21Feb/Transform/transformed_examples/26, id: 31)]
        ))

In [36]:
import pprint
import tensorflow_data_validation as tfdv

tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = transform.outputs['transformed_examples'].get()[0].uri + '/train'

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a TFRecordDataset to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
decoder = tfdv.TFExampleDecoder()

# Iterate over the first 3 records and decode them using a TFExampleDecoder
for tfrecord in dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = decoder.decode(serialized_example)
    pp.pprint(example)

{'consumer_disputed_xf': array([0.], dtype=float32),
 'state_xf': array([2]),
 'sub_product_xf': array([], dtype=int64),
 'zip_code_xf': array([395])}


In [41]:
#trainer_file = os.path.join(base_dir, '../components/keras_trainer.py')

trainer_file = './keras_trainer.py'

#trainer_file = './keras_trainer_simple.py'


In [42]:
from tfx.components.trainer.component import Trainer
from tfx.proto import trainer_pb2
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor

TRAINING_STEPS = 200
EVALUATION_STEPS = 100

trainer = Trainer(
    module_file=trainer_file,
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))
context.run(trainer)

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-24T23_22_02.736385"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-24T23_22_02.736385"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-24T23_22_02.736385');


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sub_product_xf (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
state_xf (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
zip_code_xf (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
company_xf (InputLayer)         [(None, 1)]          0                                            
____________________________________________________________________________________________

ExecutionResult(
    component_id: Trainer
    execution_id: 27
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: ../tfx/21Feb/Trainer/model/27, id: 32)]
        ))

In [43]:
import tensorflow_model_analysis as tfma
from tfx.components.evaluator.component import Evaluator

model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
        metrics_specs=tfma.metrics.default_binary_classification_specs()
        ))
context.run(model_analyzer)

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-24T23_22_02.736385"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-24T23_22_02.736385"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-24T23_22_02.736385');


ExecutionResult(
    component_id: Evaluator
    execution_id: 28
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(type_name: ModelEvaluation, uri: ../tfx/21Feb/Evaluator/evaluation/28, id: 33)]
        ))

In [44]:
context.show(model_analyzer.outputs['evaluation'])

In [45]:

import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = model_analyzer.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='product')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'product:Mortgage', '…

In [ ]:
from tfx.components.model_validator.component import ModelValidator

model_validator = ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'])
context.run(model_validator)

In [ ]:
#mkdir

In [ ]:
!mkdir -p /tmp/serving_model_dir

from tfx.components.pusher.component import Pusher
from tfx.proto import pusher_pb2

_serving_model_dir = "/tmp/serving_model_dir"

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

In [ ]:
!cat /tmp/serving_model_dir/1580932079/assets/vocab_compute_and_apply_vocabulary_1_vocabulary